In [ ]:
# Application of FL task
from MLModel import *

from utils import *
from rdp_analysis import calibrating_sampled_gaussian
from privacy_analysis import calibrating_distp_noise

from torchvision import datasets, transforms
import torch
import numpy as np
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

data_set = 'mnist'

In [ ]:
"""
1. load_data
2. generate clients (step 3)
3. generate aggregator
4. training
"""
client_num = 4
d = torch.load('scattered-mnist-noniid.pt')

torch.cuda.empty_cache()

In [ ]:
output_size = 10
T = 5
q = 0.05
lr = 0.1
clip = 0.01

In [ ]:
"""
FL model parameters.
"""
from FLModel_row import *

import time
import warnings
warnings.filterwarnings("ignore")

for E in [5, 22, 39, 59, 81, 103, 133]:
    avg_acc = []
    for _ in range(5):
        fl_param = {
            'output_size': output_size,
            'client_num': client_num,
            'model': 'scatter',
            'data': d,
            'lr': lr,
            'E': E,
            'C': 1,
            'q': q,
            'clip': clip,
            'noise': 10.0,
            'm': 32,
            'tot_T': T,
            'batch_size': 128,
            'device': device,
            'pth': "model/tune/"
        }

        fl_entity = FLServer(fl_param).to(device)
        acc = []
        start_time = time.time()
        for t in range(fl_param['tot_T']):
            acc += [fl_entity.global_update(t+1)]
        avg_acc.append(acc[-1])
        print("acc = {:.4f}".format(acc[-1]))
    print("E = {}, avg acc = {}".format(E, np.mean(avg_acc)))